## Supply Chain Risk Management Models

### 経済発注量モデル


発注量 $Q$

サイクル時間（発注周期）$T$ （$Q=d T$ の関係）

総費用はサイクル時間 $T$ の関数：

$$
  f(T)= \frac{K +h d T^2/2  +b d \beta(T) /\mu }{T +\beta(T)/\mu }
$$

$$
  \beta (T)= \frac{\lambda}{\lambda +\mu} \left( 1- \exp^{-(\lambda+\mu) T}   \right) 
$$


$\beta (T)$ は在庫が $0$ になったときに途絶している確率

$1/\mu$ は途絶期間の期待値であるので，$f(T)$ の分母の $T +\beta(T)/\mu$ は1周期の期待値



In [11]:
from amplpy import AMPL, Environment, DataFrame,  register_magics
ampl = AMPL(Environment(r'/Applications/ampl_macosx64/'))
register_magics(store_name='_ampl_cells', ampl_object=ampl)

In [14]:
%%ampl_eval
# EOQ Model with Disruption
# Copyright Log Opt Co., Ltd.
# File Name = eoq1.MOD
reset;
param N := 3; #number of stages
set Stages:=1..N;
param K;                    
param d; 
param h{Stages} default 0.;  
param b{Stages} default 100.0;  
param lambda_;
param mu ;  

var T{Stages}>=0;      #cycle time
var beta{Stages} >=0;  #disruption probability when inventory reaches 0 
var gamma{Stages} >=0; #disruption probability with which an upstream stage fails

#minimize Cost: sum{i in Stages} 
#    (K+h[i]*d*T[i]*T[i]/2+b[i]*d*beta[i]/mu)/(T[i]+beta[i]/mu) ;

minimize Cost: sum{i in Stages} 
    (K+h[i]*d*T[i]*T[i]/2+b[i]*d*gamma[i]/mu)/(T[i]+gamma[i]/mu) ;
    
subject to con1{i in Stages}:
     beta[i]=lambda_/(lambda_+mu)*(1.0-exp(-(lambda_+mu)*T[i]));

subject to con2{i in Stages}:
     gamma[i]=1.0-  (prod{j in i..N} (1-beta[j]));

subject to connect{i in 1..N-1}:
     T[i] <= T[i+1];
data;
param K:=300;
param d:=100; 
#param h:=10;  
#param b:=100;
param lambda_ :=0.05;
param mu     := 0.2;  

In [16]:
%%ampl_eval
option solver knitro;
for{i in Stages}{
  let h[i] := (N+1-i)*10.0;
}
solve;
#option display_1col 50;
#option display_round 10;
display T;
for{i in Stages}{
  #display lambda/(lambda+mu),(1.0-exp(-(lambda+mu)*T[i]));
  #display lambda/(lambda+mu)*(1.0-exp(-(lambda+mu)*T[i]));
  display i,beta[i],gamma[i];
}

Artelys Knitro 12.1.0:Knitro 12.1.0: Locally optimal solution.
objective 11271.37049; feasibility error 8.8e-11
0 iterations; 1 function evaluations
T [*] :=
1  0.718038
2  0.718038
3  0.718038
;

i = 1
beta[i] = 0.032864
gamma[i] = 0.0953874

i = 2
beta[i] = 0.032864
gamma[i] = 0.0646479

i = 3
beta[i] = 0.032864
gamma[i] = 0.032864



### 途絶を考慮した新聞売り子モデル

- $h$: 新聞 $1$部が売れ残ったときに課せられる在庫費用．
- $b$: 新聞 $1$部が品切れしたときに課せられる品切れ費用．
- $D(n)$: 需要が定常であると仮定したときの，$n$期分の需要量の合計を表す確率変数．
- $\alpha$: 調達先が途絶する確率．
- $\beta$: 途絶から回復する確率． 

推移行列
$$
\left(
\begin{array}{cc}
1-\alpha & \alpha \\
\beta & 1-\beta \\
\end{array}
\right)
$$

定常分布
- $u$: up
- $d$: down 
$$
(u,d)= \left( \frac{\beta}{\alpha + \beta }, \frac{\alpha}{\alpha + \beta } \right)
$$



### 動的価格付けを考慮した多期間在庫モデル

In [17]:
ampl = AMPL(Environment(r'/Applications/ampl_macosx64/'))
register_magics(store_name='_ampl_cells', ampl_object=ampl)

In [19]:
%%ampl_eval
# Newsboy model with pricing (multi-perid static model)
# Copyright Log Opt Co., Ltd.
# File Name = newsboy3.MOD

param mu;
param sigma;
param S ;  # number of scenarios
param T ;  # number of periods 
param h ;  # inventory holding cost
param b ;  # backorder cost

set Period :={1..T};
set Scenario :={1..S};
param Eps{Scenario,Period};
param Prob{Scenario};

var x{Period}>=0; #ordering amount
var P{Period}>=0; #price 
var D{Period}>=0; #basic demand
var Demand{Scenario,Period}>=0; #actual demand
var Inv{Scenario,Period}>=0;
var Backorder{Scenario,Period}>=0;

maximize TotalRevenue: sum{t in Period} sum{s in Scenario} 
  Prob[s]*(Demand[s,t]*P[t] -h*Inv[s,t]-b*Backorder[s,t]);

subject to DemConnect{s in Scenario, t in Period}:
  #Demand[s,t] =D[t] +Eps[s,t];  # additive model 
  Demand[s,t] =Eps[s,t]*D[t];   # multiplicative model

subject to Flow{s in Scenario,t in Period}: 
    (if t=1 then 0  else Inv[s,t-1])+
     x[t]-Inv[s,t]+Backorder[s,t] = Demand[s,t];
    
subject to DemPrice{t in Period}:
    D[t] = mu-P[t];
    
data;
param mu    :=100;
param sigma :=10;
param S :=1000;  # number of scenarios 
param T :=1; 
param h :=1;   # inventory holding cost
param b :=100;   # backorder cost


In [21]:
%%ampl_eval
option solver knitro;

printf "Perod,Sigma,P,D,x,TotalRevenue\n" > newsboy5.csv ;

for{i in 0..10}{
 #let sigma:= i; #additive case
 let sigma := i*0.1;
 option seed 1;
 for{t in Period}{
  for{s in Scenario}{
  #let Eps[s,t] :=  Normal(0,sigma); #additive model
  let Eps[s,t] :=  max(Normal(1,sigma),0); #multiplicative model
  let Prob[s]   :=1.0/S;
  }
}     
 solve;
 printf  {t in Period}: "%3i,%3i, %6.2f, %6.2f, %6.2f, %6.2f \n",
    t,i,P[t],D[t],x[t],TotalRevenue
    > newsboy5.csv; 
}
close newsboy5.csv;

Artelys Knitro 12.1.0:Knitro 12.1.0: Locally optimal solution.
objective 2499.99888; feasibility error 4.26e-14
2 iterations; 0 function evaluations
Artelys Knitro 12.1.0:Knitro 12.1.0: Locally optimal solution.
objective 2483.785974; feasibility error 7.74e-12
12 iterations; 0 function evaluations
Artelys Knitro 12.1.0:Knitro 12.1.0: Locally optimal solution.
objective 2428.163334; feasibility error 7.75e-12
10 iterations; 0 function evaluations
Artelys Knitro 12.1.0:Knitro 12.1.0: Locally optimal solution.
objective 2446.790937; feasibility error 4.24e-11
11 iterations; 0 function evaluations
Artelys Knitro 12.1.0:Knitro 12.1.0: Locally optimal solution.
objective 2478.046512; feasibility error 6.05e-12
14 iterations; 0 function evaluations
Artelys Knitro 12.1.0:Knitro 12.1.0: Locally optimal solution.
objective 2459.596716; feasibility error 2.4e-11
15 iterations; 0 function evaluations
Artelys Knitro 12.1.0:Knitro 12.1.0: Locally optimal solution.
objective 2429.268732; feasibility

In [ ]:
# Newsboy model with pricing (multi-perid static CVaR model)
# Copyright Log Opt Co., Ltd.
# File Name = newsboycvar1.MOD

param mu ;
param sigma;
param S ;  # number of scenarios
param T ;  # number of periods 
param h ;  # inventory holding cost
param b ;  # backorder cost


set Period :={1..T};
set Scenario :={1..S};
param Eps{Scenario,Period};
param Prob{Scenario};

var x{Period}>=0; #ordering amount
var P{Period}>=0; #price 
var D{Period}>=0; #basic demand
var Demand{Scenario,Period}>=0; #actual demand
var Inv{Scenario,Period}>=0;
var Backorder{Scenario,Period}>=0;

param beta;
var f{Scenario};
var V{Scenario}>=0;
var alpha;

minimize CVaR: 
alpha + 1.0/(1.0-beta)*(sum{s in Scenario} Prob[s]*V[s]);

subject to CVaR1{s in Scenario}: 
f[s] = sum{t in Period} 
      (-Demand[s,t]*P[t] +h*Inv[s,t]+b*Backorder[s,t]) ;

subject to CVaR2{s in Scenario}:
V[s] >= f[s] - alpha;

subject to DemConnect{s in Scenario, t in Period}:
  Demand[s,t] =D[t] +Eps[s,t];  # additive model 
  #Demand[s,t] =Eps[s,t]*D[t];   # multiplicative model

subject to Flow{s in Scenario,t in Period}: 
    (if t=1 then 0  else Inv[s,t-1])+
     x[t]-Inv[s,t]+Backorder[s,t] = Demand[s,t];
    
subject to DemPrice{t in Period}:
    D[t] = mu-P[t];
    
data;
param mu    :=100;
param sigma :=10;
param S :=1000;  # number of scenarios 
param T :=1; 
param h :=1;   # inventory holding cost
param b :=100;   # backorder cost

param beta :=0.5;